# Reanimação Neonatal em Neonatologia
Projeto 1 da Disciplina de Aprendizado de Máquina - PPgEEC/UFRN

## Etapa 1 - ETL

Este projeto tem como objetivo desenvolver um *pipeline* para deposição de um modelo classificatório em um ambiente de produção.

O modelo se propõe a resolver um problema de **classificação** que tenta separar os recém nascidos que possam necessitar de *Reanimação Neonatal* baseado nas seguintes variáveis(*labels*):

* Idade materna
* Fumo na gravidez
* Uso de álcool na gravidez
* Uso de drogas psicoativas na gravidez
* Trabalho de parto prematuro
* Descolamento prematuro de placenta
* Oligoâmnio
* Doença Hipertensiva Específica da Gravidez(DHEG)
* Diabetes Mellitus Gestacional(DMG)
* HIV
* Sífilis
* Covid
* Sexo do recém-nascido
* Índice de Apgar no primeiro minuto de vida
* Reanimação neonatal

Os dados para este modelo serão fornecidos pelo banco de dados da uma UTI neonatal do Estado após prévia autorização por escrito do responsável pelas informações.

## 1. Configuração do Ambiente de Trabalho

In [1]:
# Importações

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
import requests
import json
from pandas_profiling import ProfileReport

In [2]:
# Login no wandb
!wandb login c4d070b0b80b59891761b50eb9912749a4af585e --relogin

wandb: Appending key for api.wandb.ai to your netrc file: /Users/ericcalasans/.netrc


## 2.  Etapas do ETL

### 2.1. Aquisição dos Dados(*Fetch Data*)

Os dados são provenientes da plataforma https://neo.redcaprn.org através de API própria da plataforma

In [30]:
# Conexão com a base de dados e aquisição
data = {
    'token': '4276F33FF3B3F9E3CEA6A50EFAD0C213',
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://neo.redcaprn.org/api/',data=data)
r_dump = json.dumps(r.json())
df = pd.read_json(r_dump, dtype=True, convert_dates=True)

O *dataset* consiste de várias tabelas do qual são selecionadas as tabelas de interesse das quais serão escolhidos os *labels*:

In [33]:
ident = df[['nome_pac', 'nascido_hsc', 'setor_origem_pac', 'idade_materna',
            'cor_pele', 'proc_materna', 'escolaridade_materna', 'trabalho_mae',
            'gesta', 'abortos', 'filhos_vivos', 'natimortos', 'mortos_menor_1_ano',
            'ts_mae', 'cons_prenatal']]

ant_maternos = df[['fumo', 'alcool', 'freq_alcool', 'psicoativas', 'violencia',
                   'corioamnionite', 'rcui', 'tpp', 'dpp', 'has', 'dheg', 'hellp',
                   'sulfatada', 'oligoamnio', 'dm', 'hiv', 'hep_b', 'hep_c', 'sifilis',
                   'toxoplasmose', 'cmv', 'rubeola', 'covid_mae',
                   'outras_doencas_maternas', 'cort_antenatal']]

parto = df[['data_nasc', 'sexo', 'tempo_br', 'tipo_parto', 'gemelar',
            'apgar_1_minuto', 'apgar_5_min', 'reanimacao', 'o2_21',
            'massagem_cardiaca', 'cpap_sp', 'drogas_reanimacao', 'liq_amniotico',
            'clamp_cu', 'ev_hipotermia', 'saco_plastico', 'touca', 'colchao']]

Da tabela **resp** vem a variável *target* **ttrn**:

Formação do *dataset* bruto(*raw* dataset) através da seleção das *labels* e *target*:

In [52]:
rean_df = pd.concat(
    [
        ident['idade_materna'],
        ant_maternos['fumo'],
        ant_maternos['alcool'],
        ant_maternos['psicoativas'],
        ant_maternos['tpp'],
        ant_maternos['dpp'],
        ant_maternos['oligoamnio'],
        ant_maternos['sifilis'],
        ant_maternos['hiv'],
        ant_maternos['covid_mae'],
        ant_maternos['dheg'],
        ant_maternos['dm'],
        parto['sexo'],
        parto['apgar_1_minuto'],
        parto['reanimacao'] #Target
    ],
    axis=1
)

Criação do artefato inicial

In [53]:
#  Salva como .csv
rean_df.to_csv('rean_raw.csv', index=False)

In [55]:
# Cria o artefato
!wandb artifact put --name mlreanimacao/rean_raw.csv --type rean_raw --description "Reanimação - Dados brutos" rean_raw.csv

wandb: Uploading file rean_raw.csv to: "ecalasans/mlreanimacao/rean_raw.csv:latest" (rean_raw)
wandb: Currently logged in as: ecalasans (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.15
wandb: Run data is saved locally in /Users/ericcalasans/Documents/Projetos/ml_proj1/wandb/run-20220517_160829-2wb5873c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run golden-water-2
wandb: ⭐️ View project at https://wandb.ai/ecalasans/mlreanimacao
wandb: 🚀 View run at https://wandb.ai/ecalasans/mlreanimacao/runs/2wb5873c
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("ecalasans/mlreanimacao/rean_raw.csv:v0")

wandb: Waiting for W&B process to finish... (success).


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/ecalasans/mlttrn/rvade8vy/file_stream
NoneType: None


wandb:                                                                                
wandb: Synced golden-water-2: https://wandb.ai/ecalasans/mlreanimacao/runs/2wb5873c
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20220517_160829-2wb5873c/logs


### 2.2. Análise Exploratória(*EDA*)

Primeira impressão do *dataset*

In [57]:
rean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idade_materna   583 non-null    object
 1   fumo            583 non-null    object
 2   alcool          583 non-null    object
 3   psicoativas     583 non-null    object
 4   tpp             583 non-null    object
 5   dpp             583 non-null    object
 6   oligoamnio      583 non-null    object
 7   sifilis         583 non-null    object
 8   hiv             583 non-null    object
 9   covid_mae       583 non-null    object
 10  dheg            583 non-null    object
 11  dm              583 non-null    object
 12  sexo            583 non-null    object
 13  apgar_1_minuto  583 non-null    object
 14  reanimacao      583 non-null    object
dtypes: object(15)
memory usage: 68.4+ KB


Aparentemente não há objetos nulos mas isto é explicado pelo fato dos dados se apresentarem como sendo do tipo *string(object)* e os valores que possam não estar preenchidos se apresentem como *strings* vazias.  Uma verificação mais detalhada mostra a presença de dados nulos/ausentes nas *labels*.

In [3]:
rean_df.sexo.unique()

NameError: name 'rean_df' is not defined

### 2.3. Preprocessamento(*Preprocessing*) e Limpeza de Dados(*Cleaning Data*)

Captura de artefato do wandb

In [19]:
run = wandb.init(project='mlreanimacao', job_type='process_data')

In [20]:
# Uso do artefato
rean_artifact = wandb.use_artifact('mlreanimacao/rean_raw.csv:latest')
df_rean = pd.read_csv(rean_artifact.file())

In [21]:
# Verificação
df_rean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   idade_materna   563 non-null    float64
 1   fumo            490 non-null    float64
 2   alcool          489 non-null    float64
 3   psicoativas     445 non-null    float64
 4   tpp             579 non-null    float64
 5   dpp             579 non-null    float64
 6   oligoamnio      577 non-null    float64
 7   sifilis         579 non-null    float64
 8   hiv             580 non-null    float64
 9   covid_mae       571 non-null    float64
 10  dheg            579 non-null    float64
 11  dm              576 non-null    float64
 12  sexo            582 non-null    float64
 13  apgar_1_minuto  555 non-null    float64
 14  reanimacao      576 non-null    float64
dtypes: float64(15)
memory usage: 68.4 KB


De acordo com o dicionário de dados fornecido pela base de dados, as *labels* e a *target* são classificadas como:

* **Categóricas** - *fumo*, *álcool*, *psicoativas*, *tpp*, *dheg*, *dm*, *sexo*, *reanimacao*, *rean*, *convulsao*
* **Numéricas** - *idade_materna*, *peso_nasc*, *estatura*, *pc*


In [22]:
df_rean.sexo.unique()

array([ 2.,  1., nan,  3.])

Percebe-se grande quantidade de dados faltantes em algumas *labels*.

Em consulta ao dicionário de dados tem-se as seguintes convenções para as *labels* e *target*:

* **idade_materna** - Numérico, em anos;
* **fumo** - Sim(1), Não(2)
* **alcool** - Sim(1), Não(2)
* **psicoativas** - Sim(1), Não(2)
* **tpp** - Sim(1), Não(2), Desconhecido(3)
* **dpp** - Sim(1), Não(2), Desconhecido(3)
* **oligoamnio** - Sim(1), Não(2), Desconhecido(3)
* **dheg** - Sim(1), Não(2), Desconhecido(3)
* **dm** - Não(1), Prévio, não insulinodependente(2), Prévio, insulinodependente(3), Gestacional, não insulinodependente(4), Gestacional, insulinodependente(5);
* **sifilis** - Sim(1), Não(2), Desconhecido(3)
* **hiv** - Sim(1), Não(2), Desconhecido(3)
* **covid_mae** - Sim(1), Não(2), Desconhecido(3)
* **sexo** - Feminino(1), Masculino(2), Indefinido(3);
* **apgar_1_minuto** - Numérico;
* **reanimacao** - Não(1), O2 inalatório(2), VPP com balão e máscara(3), VPP com IOT(4);


A título de adequação, a *feature* **dm** será consolidada como *Sim* e *Não*, bem como a *target*, posteriormente.

Contagem de NaN's

In [23]:
df_rean.isna().sum()

idade_materna      20
fumo               93
alcool             94
psicoativas       138
tpp                 4
dpp                 4
oligoamnio          6
sifilis             4
hiv                 3
covid_mae          12
dheg                4
dm                  7
sexo                1
apgar_1_minuto     28
reanimacao          7
dtype: int64

As *labels* categóricas ausentes serão preenchidas com o valor **Não** e as numéricas com o valor médio.

In [24]:
#  Preenchimento
#  Labels
df_rean['idade_materna'].fillna(
    np.round(df_rean['idade_materna'].mean(), 0),
    inplace=True
)

df_rean['fumo'].fillna(2, inplace=True)
df_rean['alcool'].fillna(2, inplace=True)
df_rean['psicoativas'].fillna(2, inplace=True)
df_rean['tpp'].fillna(2, inplace=True)
df_rean['dheg'].fillna(2, inplace=True)
df_rean['dm'].fillna(2, inplace=True)

df_rean['apgar_1_minuto'].fillna(
    np.round(df_rean['apgar_1_minuto'].mean(), 0),
    inplace=True)

df_rean['reanimacao'].fillna(1, inplace=True)
df_rean['sexo'].fillna(3, inplace=True)
df_rean['oligoamnio'].fillna(2, inplace=True)
df_rean['dpp'].fillna(2, inplace=True)
df_rean['sifilis'].fillna(2, inplace=True)
df_rean['hiv'].fillna(2, inplace=True)
df_rean['covid_mae'].fillna(2, inplace=True)

In [25]:
df_rean.drop(df_rean[df_rean['sexo'] == 3].index, inplace=True)

In [26]:
df_rean.sexo.unique()

array([2., 1.])

Na sequência, os valores numéricos das *labels* categóricas serão substituídos por seus correspondentes de texto, de acordo com o dicionário de dados.

In [27]:
df_rean['fumo'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['alcool'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['psicoativas'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['tpp'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['dheg'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)

# Melhor adequação da feature sexo
df_rean['sexo'].replace([1,2,3], ['Feminino', 'Masculino','Indefinido'], inplace=True)
df_rean['dpp'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['oligoamnio'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['sifilis'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['hiv'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)
df_rean['covid_mae'].replace([0, 1, 2, 3], ['Não', 'Sim', 'Não', 'Desconhecido'], inplace=True)

# Consolidações
df_rean['dm'].replace([0,1,2,3,4,5,6], ['Não', 'Não','Sim','Sim', 'Sim','Sim','Desconhecido'], inplace=True)
df_rean['reanimacao'].replace([0, 1, 2, 3, 4, 5], ['Não', 'Não', 'Não', 'Sim', 'Sim', 'Não'], inplace=True)

In [27]:
df_rean['fumo'].replace([0, 1, 2, 3], ['n_fumo', 's_fumo', 'n_fumo', 'd_fumo'], inplace=True)
df_rean['alcool'].replace([0, 1, 2, 3], ['n_alcool', 's_alcool', 'n_alcool', 'd_alcool'], inplace=True)
df_rean['psicoativas'].replace([0, 1, 2, 3], ['n_psico', 's_psico', 'n_psico', 'd_psico'], inplace=True)
df_rean['tpp'].replace([0, 1, 2, 3], ['n_tpp', 's_tpp', 'n_tpp', 'd_tpp'], inplace=True)
df_rean['dheg'].replace([0, 1, 2, 3], ['n_dheg', 's_dheg', 'n_dheg', 'd_dheg'], inplace=True)

# Melhor adequação da feature sexo
df_rean['sexo'].replace([1,2,3], ['Feminino', 'Masculino','Indefinido'], inplace=True)
df_rean['dpp'].replace([0, 1, 2, 3], ['n_dpp', 's_dpp', 'n_dpp', 'd_dpp'], inplace=True)
df_rean['oligoamnio'].replace([0, 1, 2, 3], ['n_oligo', 's_oligo', 'n_oligo', 'd_oligo'], inplace=True)
df_rean['sifilis'].replace([0, 1, 2, 3], ['n_sifilis', 's_sifilis', 'n_sifilis', 'd_sifilis'], inplace=True)
df_rean['hiv'].replace([0, 1, 2, 3], ['n_hiv', 's_hiv', 'n_hiv', 'd_hiv'], inplace=True)
df_rean['covid_mae'].replace([0, 1, 2, 3], ['n_covid', 's_covid', 'n_covid', 'd_covid'], inplace=True)

# Consolidações
df_rean['dm'].replace([0,1,2,3,4,5,6], ['n_dm', 'n_dm','s_dm','s_dm', 's_dm','s_dm','d_dm'], inplace=True)
df_rean['reanimacao'].replace([0, 1, 2, 3, 4, 5], ['nr', 'nr', 'nr', 'sr', 'sr', 'nr'], inplace=True)

In [28]:
df_rean.reanimacao.unique()

array(['nr', 'sr'], dtype=object)

Perfil do dataset após preprocessamento

In [29]:
ProfileReport(df_rean, title="Profile de df_rean", explorative=True)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Pelo perfilamento percebe-se uma alta correlação entre o Apgar no primeiro minuto e a reanimação neonatal.  As outras *features* apresentam uma correlação fraca.  No entanto o *dataset* será utilizado.

Percebe-se também a presença de dados duplicados.  Explica-se isto pelo fato de poder haver mais de uma paciente com as mesmas características(*features*).  Para efeito de treinamento do algoritmo, a presença destes dados duplicados pode implicar em aumento do tempo de execução do treinamento e, desta forma, serão excluídos do *dataset*

In [29]:
df_rean.drop_duplicates(inplace=True, ignore_index=True)

In [30]:
df_rean.columns

Index(['idade_materna', 'fumo', 'alcool', 'psicoativas', 'tpp', 'dpp',
       'oligoamnio', 'sifilis', 'hiv', 'covid_mae', 'dheg', 'dm', 'sexo',
       'apgar_1_minuto', 'reanimacao'],
      dtype='object')

In [14]:
ProfileReport(df_rean, title="Profile de checagem após limpeza de duplicatas", explorative=True)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Criação do artefato e upload para o **wandb**

In [31]:
artifact_name = 'clean_data.csv'
df_rean.to_csv(artifact_name, index=False)

In [32]:
artifact = wandb.Artifact(
    name=artifact_name,
    type='clean_data',
    description='Dados preprocessados e limpos'
)
artifact.add_file(artifact_name)

<ManifestEntry digest: ar8geLyv/gO47ouR0AaktQ==>

In [33]:
# Upload
run.log_artifact(artifact)

In [34]:
# Finaliza wandb
run.finish()